# Assignment 4
Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from assets/wikipedia_data.html and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in assets/nfl.csv), MLB (baseball, in assets/mlb.csv), NBA (basketball, in assets/nba.csv or NHL (hockey, in assets/nhl.csv). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: what is the win/loss ratio's correlation with the population of the city it is in? Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with pearsonr, so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%*4=80%) of the grade for this assignment. You should only use data from year 2018 for your analysis -- this is important!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NHL using 2018 data.

In [9]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# function to clean cities name from nhl file
def clean_city_1(city):
    area = city.split()[0]
    
    # Some names need to be updated manually
    if area in ['New','Los','St.','San','Tampa']: area = area + ' ' + city.split()[1]
    if area == "Vegas": area = 'Las Vegas'
    if area == "New Jersey": area = 'New York'
    if area == "San Jose": area = 'San Francisco'
    if area == "Arizona": area = 'Phoenix'
    if area == "Florida": area = 'Miami'
    if area == "Anaheim": area = 'Los Angeles'
    if area == "Colorado": area = 'Denver'
    if area == "Carolina": area = 'Raleigh'
    if area == "Minnesota": area = 'Minneapolis'
    return area

# function to clean cities name from wikipedia data
def clean_city_2(city):
    # Some names need to be updated manually
    dct = {'New York City':'New York',
          'San Francisco Bay Area':'San Francisco',
          'Washington, D.C.':'Washington',
          'Tampa Bay Area':'Tampa Bay'
          }   
    return dct.get(city,city)



def nhl_correlation(): 
    # YOUR CODE HERE
    # load files
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # clean df
    nhl_df = (nhl_df[nhl_df['year'] == 2018].replace('\*','',regex=True)[['team','W','L']][~nhl_df['team'].str.contains("Division")])
    nhl_df = nhl_df.astype({'W': 'int32','L': 'int32' })
    nhl_df['area'] = nhl_df['team'].apply(clean_city_1)
    # group by 'area' and get ratio mean
    nhl_df['ratio'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])
    nhl_df = nhl_df.groupby('area').agg({'ratio':np.mean})
    #clean df cities
    cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    cities = cities.replace("\[.*\]","", regex=True).replace(r'^\s*$', np.nan, regex=True).replace(r'—', np.nan, regex=True).replace(r'\s$',"",regex=True).replace(r'\s+',' ',regex=True).replace(r'–\w+\s\w+','',regex=True)
    cities['area'] = cities['Metropolitan area'].apply(clean_city_2)
    cities = cities.astype({'Population': 'int32'})
    # group cities by 'area' and sum 'Population'
    cities = cities.groupby('area').agg({'Population':np.sum})

    # merge 
    df = pd.merge(cities, nhl_df, how='inner',left_index=True, right_index=True).dropna()
    #raise NotImplementedError()
    
    population_by_region = df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = df['ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


C:\Users\imont\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


0.012486162921209923

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NBA using 2018 data.

In [10]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# function to clean team name
def clean_area(city):
    area = city.split()[0]
    
    # Some names need to be updated manually
    if area in ['New','Los','St.','San','Tampa', 'Oklahoma']: area = area + ' ' + city.split()[1]
    if area == "Vegas": area = 'Las Vegas'
    if area in ["New Jersey", "Brooklyn"]: area = 'New York'
    if area in ["San Jose", "Golden"]: area = 'San Francisco'
    if area == "Arizona": area = 'Phoenix'
    if area == "Florida": area = 'Miami'
    if area == "Anaheim": area = 'Los Angeles'
    if area == "Colorado": area = 'Denver'
    if area == "Carolina": area = 'Raleigh'
    if area == "Minnesota": area = 'Minneapolis'
    if area == "Indiana": area = 'Indianapolis'
    if area == "Utah": area = 'Salt Lake City'
    return area

# function to clean cities name from wikipedia data
def clean_city_2(city):
    # Some names need to be updated manually
    dct = {'New York City':'New York',
          'San Francisco Bay Area':'San Francisco',
          'Dallas–Fort Worth':'Dallas',
          'Washington, D.C.':'Washington',
          'Minneapolis–Saint Paul':'Minneapolis',
          'Miami–Fort Lauderdale':'Miami',
          'Tampa Bay Area':'Tampa Bay'
          }   
    return dct.get(city,city)

def nba_correlation():
    # YOUR CODE HERE
    #load file
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    #clean nba data
    nba_df = nba_df[nba_df['year'] == 2018].replace("\(.*\)","", regex=True).replace('\*','',regex=True)[['team','W/L%']][~nba_df['team'].str.contains("Division")]
    nba_df = nba_df.astype({'W/L%':'float64'})
    nba_df['ratio'] = nba_df['W/L%']
    nba_df['area'] = nba_df['team'].apply(clean_area)
    nba_df = nba_df.groupby('area').agg({'ratio':np.mean})

    #clean df cities
    cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    cities = cities.replace("\[.*\]","", regex=True).replace(r'^\s*$', np.nan, regex=True).replace(r'—', np.nan, regex=True).replace(r'\s$',"",regex=True).replace(r'\s+',' ',regex=True).replace(r'–\w+\s\w+','',regex=True)
    cities['area'] = cities['Metropolitan area'].apply(clean_city_2)
    cities = cities.astype({'Population': 'int32'})
    cities.drop(['NFL','MLB','Metropolitan area','NHL'], inplace=True, axis=1)
    # group cities by 'area' and sum 'Population'
    cities = cities.groupby('area').agg({'Population':np.mean})

    # merge 
    df = pd.merge(cities, nba_df, how='inner', left_index=True, right_index=True).dropna()

    #raise NotImplementedError()
    population_by_region = df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = df['ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    #raise NotImplementedError()
    

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the MLB using 2018 data.

In [11]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# function to clean team name
def clean_area(city):
    area = city.split()[0]
    
    # Some names need to be updated manually
    if area in ['New','Los','St.','San','Tampa', 'Oklahoma', 'Kansas']: area = area + ' ' + city.split()[1]
    if area == "Vegas": area = 'Las Vegas'
    if area in ["New Jersey", "Brooklyn"]: area = 'New York'
    if area in ["San Jose", "Golden", "Oakland"]: area = 'San Francisco'
    if area == "Arizona": area = 'Phoenix'
    if area == "Florida": area = 'Miami'
    if area == "Anaheim": area = 'Los Angeles'
    if area == "Colorado": area = 'Denver'
    if area == "Carolina": area = 'Raleigh'
    if area == "Minnesota": area = 'Minneapolis'
    if area == "Indiana": area = 'Indianapolis'
    if area == "Utah": area = 'Salt Lake City'
    if area == "Texas": area = 'Dallas'
    return area

# function to clean cities name from wikipedia data
def clean_city(city):
    # Some names need to be updated manually
    dct = {'New York City':'New York',
          'San Francisco Bay Area':'San Francisco',
          'Dallas–Fort Worth':'Dallas',
          'Washington, D.C.':'Washington',
          'Minneapolis–Saint Paul':'Minneapolis',
          'Miami–Fort Lauderdale':'Miami',
          'Tampa Bay Area':'Tampa Bay'
          }   
    return dct.get(city,city)


def mlb_correlation(): 
    # YOUR CODE HERE

    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    # clean mlb df
    mlb_df = mlb_df[mlb_df['year'] == 2018].replace("\(.*\)","", regex=True).replace('\*','',regex=True)[['team','W-L%']][~mlb_df['team'].str.contains("Division")]
    mlb_df.rename(columns={'W-L%':'ratio'}, inplace=True)
    mlb_df = mlb_df.astype({'ratio':'float64'})
    mlb_df['area'] = mlb_df['team'].apply(clean_area)
    mlb_df = mlb_df.groupby('area').agg({'ratio':np.mean})

    #clean df cities
    cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    cities = cities.replace("\[.*\]","", regex=True).replace(r'^\s*$', np.nan, regex=True).replace(r'—', np.nan, regex=True).replace(r'\s$',"",regex=True).replace(r'\s+',' ',regex=True).replace(r'–\w+\s\w+','',regex=True)
    cities['area'] = cities['Metropolitan area'].apply(clean_city)
    cities = cities.astype({'Population': 'int32'})
    cities.drop(['NFL','NBA','Metropolitan area','NHL','MLB'], inplace=True, axis=1)
    cities = cities.groupby('area').agg({'Population':np.mean})

     # merge 
    df = pd.merge(cities, mlb_df, how='inner',left_index=True, right_index=True).dropna()

    #raise NotImplementedError()

    population_by_region = df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = df['ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


C:\Users\imont\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:51: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


0.15003737475409498

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NFL using 2018 data.

In [15]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# function to clean team name
def clean_area(city):
    area = city.split()[0]
    
    # Some names need to be updated manually
    if area in ['New','Los','St.','San','Tampa', 'Oklahoma', 'Kansas', 'Green']: area = area + ' ' + city.split()[1]
    if area == "Vegas": area = 'Las Vegas'
    if area in ["New Jersey", "Brooklyn"]: area = 'New York'
    if area in ["San Jose", "Golden", "Oakland"]: area = 'San Francisco'
    if area == "Arizona": area = 'Phoenix'
    if area == "Florida": area = 'Miami'
    if area == "Anaheim": area = 'Los Angeles'
    if area == "Colorado": area = 'Denver'
    if area == "Carolina": area = 'Raleigh'
    if area == "Minnesota": area = 'Minneapolis'
    if area == "Indiana": area = 'Indianapolis'
    if area == "Utah": area = 'Salt Lake City'
    if area == "Texas": area = 'Dallas'
    if area == "New England": area = 'Boston'
    if area == "Tennessee": area = 'Nashville'
    return area

# function to clean cities name from wikipedia data
def clean_city(city):
    # Some names need to be updated manually
    dct = {'New York City':'New York',
          'San Francisco Bay Area':'San Francisco',
          'Dallas–Fort Worth':'Dallas',
          'Washington, D.C.':'Washington',
          'Minneapolis–Saint Paul':'Minneapolis',
          'Miami–Fort Lauderdale':'Miami',
          'Tampa Bay Area':'Tampa Bay'
          }   
    return dct.get(city,city)


def nfl_correlation(): 
    # YOUR CODE HERE
    # load file
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nfl_df = pd.read_csv('assets/nfl.csv')
    # clean nfl df
    nfl_df = nfl_df[nfl_df['year'] == 2018].replace("\(.*\)","", regex=True).replace('\*','',regex=True)[['team','W-L%']][~nfl_df['W-L%'].str.contains("[A-Z]")]
    nfl_df.rename(columns={'W-L%':'ratio'}, inplace=True)
    nfl_df = nfl_df.astype({'ratio':'float64'})
    nfl_df['area'] = nfl_df['team'].apply(clean_area)

    #clean df cities
    cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    cities = cities.replace("\[.*\]","", regex=True).replace(r'^\s*$', np.nan, regex=True).replace(r'—', np.nan, regex=True).replace(r'\s$',"",regex=True).replace(r'\s+',' ',regex=True).replace(r'–\w+\s\w+','',regex=True)
    cities['area'] = cities['Metropolitan area'].apply(clean_city)
    cities = cities.astype({'Population': 'int32'})
    cities.drop(['NFL','NBA','Metropolitan area','NHL','MLB'], inplace=True, axis=1)

    df = pd.merge(cities, nfl_df, how='inner').dropna().groupby(['area']).agg({"Population":np.mean,"ratio":np.mean})

    population_by_region = df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = df['ratio'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    #raise NotImplementedError()

C:\Users\imont\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


0.008330914787036996

## Question 5
In this question I would like you to explore the hypothesis that given that an area has two sports teams in different sports, those teams will perform the same within their respective sports. How I would like to see this explored is with a series of paired t-tests (so use ttest_rel) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [108]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# function to clean team name
def clean_area(city):
    area = city.split()[0]
    
    # Some names need to be updated manually
    if area in ['New','Los','St.','San','Tampa', 'Oklahoma']: area = area + ' ' + city.split()[1]
    if area == "Vegas": area = 'Las Vegas'
    if area in ["New Jersey", "Brooklyn"]: area = 'New York'
    if area in ["San Jose", "Golden"]: area = 'San Francisco'
    if area == "Arizona": area = 'Phoenix'
    if area == "Florida": area = 'Miami'
    if area == "Anaheim": area = 'Los Angeles'
    if area == "Colorado": area = 'Denver'
    if area == "Carolina": area = 'Raleigh'
    if area == "Minnesota": area = 'Minneapolis'
    if area == "Indiana": area = 'Indianapolis'
    if area == "Utah": area = 'Salt Lake City'
    return area


# function to clean cities name from wikipedia data
def clean_city(city):
    # Some names need to be updated manually
    dct = {'New York City':'New York',
          'San Francisco Bay Area':'San Francisco',
          'Dallas–Fort Worth':'Dallas',
          'Washington, D.C.':'Washington',
          'Minneapolis–Saint Paul':'Minneapolis',
          'Miami–Fort Lauderdale':'Miami',
          'Tampa Bay Area':'Tampa Bay'
          }   
    return dct.get(city,city)



def sports_team_performance():
    # YOUR CODE HERE
    #raise NotImplementedError()

    mlb_df=pd.read_csv("assets/mlb.csv")
    nhl_df=pd.read_csv("assets/nhl.csv")
    nba_df=pd.read_csv("assets/nba.csv")
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]



    #clean df cities
    cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
    cities = cities.replace("\[.*\]","", regex=True).replace(r'^\s*$', np.nan, regex=True).replace(r'—', np.nan, regex=True).replace(r'\s$',"",regex=True).replace(r'\s+',' ',regex=True).replace(r'–\w+\s\w+','',regex=True)
    cities['area'] = cities['Metropolitan area'].apply(clean_city_2)
    cities = cities.astype({'Population': 'int32'})
    cities.drop(['NFL','MLB','Metropolitan area','NHL'], inplace=True, axis=1)
    # group cities by 'area' and sum 'Population'
    cities = cities.groupby('area').agg({'Population':np.mean})

    #clean nba data
    nba_df = nba_df[nba_df['year'] == 2018].replace("\(.*\)","", regex=True).replace('\*','',regex=True)[['team','W/L%']][~nba_df['team'].str.contains("Division")]
    nba_df = nba_df.astype({'W/L%':'float64'})
    nba_df['ratio'] = nba_df['W/L%']
    nba_df['area'] = nba_df['team'].apply(clean_area)
    nba_df = nba_df.groupby('area').agg({'ratio':np.mean})

    # clean nhl df
    nhl_df = (nhl_df[nhl_df['year'] == 2018].replace('\*','',regex=True)[['team','W','L']][~nhl_df['team'].str.contains("Division")])
    nhl_df = nhl_df.astype({'W': 'int32','L': 'int32' })
    nhl_df['area'] = nhl_df['team'].apply(clean_city_1)
    nhl_df['ratio'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])
    nhl_df = nhl_df.groupby('area').agg({'ratio':np.mean})

    # clean mlb df
    mlb_df = mlb_df[mlb_df['year'] == 2018].replace("\(.*\)","", regex=True).replace('\*','',regex=True)[['team','W-L%']][~mlb_df['team'].str.contains("Division")]
    mlb_df.rename(columns={'W-L%':'ratio'}, inplace=True)
    mlb_df = mlb_df.astype({'ratio':'float64'})
    mlb_df['area'] = mlb_df['team'].apply(clean_area)
    mlb_df = mlb_df.groupby('area').agg({'ratio':np.mean})

    # clean nfl df
    nfl_df = nfl_df[nfl_df['year'] == 2018].replace("\(.*\)","", regex=True).replace('\*','',regex=True)[['team','W-L%']][~nfl_df['W-L%'].str.contains("[A-Z]")]
    nfl_df.rename(columns={'W-L%':'ratio'}, inplace=True)
    nfl_df = nfl_df.astype({'ratio':'float64'})
    nfl_df['area'] = nfl_df['team'].apply(clean_area)
    nfl_df = nfl_df.groupby('area').agg({'ratio':np.mean})


    def create_df(sport):
        if sport =='NFL':
            return nfl_df
        elif sport =='NBA':
            return nba_df
        elif sport =='NHL':
            return nhl_df
        elif sport =='MLB':
            return mlb_df
        else:
            print("ERROR with intput!")

    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)


    for i in sports:
        for j in sports:
            if i != j:
                merge=pd.merge(create_df(i),create_df(j),'outer', on = ['area']).dropna()
                p_values.loc[i,j] = stats.ttest_rel(merge['ratio_x'], merge['ratio_y'])[1]


        
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan

    
    #assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    #assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values